# Multi-Class Tensorflow Model Trainer

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam



def count_subdirs_and_get_names(directory):
    """
    Count the number of subdirectories in the given directory and assign their names to a list.

    Parameters:
    directory (str): The path to the main directory.

    Returns:
    int: The number of subdirectories.
    list: A list containing the names of the subdirectories.
    """
    # Get the list of subdirectories
    subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    # Count the number of subdirectories
    num_subdirs = len(subdirs)
    
    return num_subdirs, subdirs



## Variables to Edit

In [ ]:
epochs = 30
# Best result 100

batch_size = 256
#Best result 256

# Set image size for model training
pic_size = 244

#Set the data directory for training
main_directory = './data_directory'

# Load and preprocess the data
train_data_dir = main_directory+'/train/'
validation_data_dir = main_directory+'/validation/'

num_classes, lbl_classes = count_subdirs_and_get_names(train_data_dir)

model_name = 'model/'+str(epochs)+'_tf_model.keras'

In [ ]:
# Sort classes alpha
lbl_classes
print(f"pic size: {pic_size} X {pic_size}")
print(f'Batch Size: {batch_size}')
print(f'Number of Epochs: {epochs}')
print(f'Number of classes: {num_classes}')
print(f'Class labels::{lbl_classes}')

## Display Dataset balance

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
from natsort import natsorted

def count_images_in_subdirs(main_dir):
    labels = []
    counts = []

    # Loop through each subdirectory in the main directory
    for subdir in os.listdir(main_dir):
        subdir_path = os.path.join(main_dir, subdir)
        if os.path.isdir(subdir_path):
            # Count the number of images in the subdirectory
            image_files = natsorted(glob.glob(f"{subdir_path}/*.jpg"))
            num_images = len(image_files)
            labels.append(subdir)
            counts.append(num_images)
    
    return labels, counts

def generate_pie_chart(labels, counts, locf='Train'):
    myexplode = [0.1] * len(labels)  # Adjust this list if you want specific slices to be exploded
    
    # Combine labels and counts for legend
    legend_labels = [f"{label} ({count})" for label, count in zip(labels, counts)]
    
    fig, ax = plt.subplots(figsize=(12.8, 9.6))  # Make the chart 2x larger
    ax.pie(counts, labels=labels, autopct='%1.1f%%',
           colors=plt.cm.tab20.colors, explode=myexplode, shadow=True, startangle=90)
    plt.legend(legend_labels, loc='upper right', title=locf + " Image Count")
    plt.title(f"{locf} - Image Distribution")
    plt.savefig('tf_'+locf+'_dataset_pie.png')
    plt.show()
    


In [ ]:
labels, counts = count_images_in_subdirs(train_data_dir)
generate_pie_chart(labels, counts, locf='Train')

In [ ]:
labels, counts = count_images_in_subdirs(validation_data_dir)
generate_pie_chart(labels, counts, locf='Validation')

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

def display_random_images_from_subdirectories(root_dir, num_images_to_display=9):
    all_images = []

    # Walk through all directories and subdirectories
    for subdir, _, files in os.walk(root_dir):
        images = [os.path.join(subdir, f) for f in files if os.path.isfile(os.path.join(subdir, f)) and f.lower().endswith('.jpg')]
        all_images.extend([(subdir, img) for img in images])

    num_images = len(all_images)
    print(f"Total number of images found: {num_images}")

    if num_images < num_images_to_display:
        print("Not enough images to display.")
        return

    # Randomly select images from the accumulated list
    random_images = random.sample(all_images, num_images_to_display)

    fig, axs = plt.subplots(3, 3, figsize=(12, 12))

    for i in range(num_images_to_display):
        subdir, img_path = random_images[i]
        img = Image.open(img_path)
        
        ax = axs[i // 3, i % 3]
        if img.mode in ['L', 'P']:  # Check if the image is grayscale
            ax.imshow(img, cmap='gray')
        else:
            ax.imshow(img)
        ax.set_title(os.path.basename(subdir))
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Example usage

display_random_images_from_subdirectories(main_directory)


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(pic_size, pic_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    classes=lbl_classes)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(pic_size, pic_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    classes=lbl_classes)

# Build the model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(pic_size, pic_size, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Function to dynamically adjust the learning rate during model training
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr  
    else:
        return float(lr * tf.math.exp(-0.1)) 

# Create a learning rate scheduler callback
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model with the learning rate scheduler callback
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[lr_scheduler_callback]  
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

# Save the model
model.save(model_name)

## Generate Model Training Metrics Chart

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Create a figure with subplots for accuracy and loss
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot training & validation accuracy values
    axs[0].plot(history.history['accuracy'])
    axs[0].plot(history.history['val_accuracy'])
    axs[0].set_title('Model accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('Model loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Validation'], loc='upper left')
    
    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the combined figure
    plt.savefig('tf_model_metrics.png')
    
    # Show the combined plot
    plt.show()

# Example usage:
# After training the model using the code you provided,
# you can call this function passing the history object as follows:
plot_training_history(history)


### Explanation of the Metrics in the Code

The provided function visualizes the training history of a machine learning model, specifically focusing on accuracy and loss over the training epochs. Here’s a simplified explanation of each part:

#### Function: `plot_training_history(history)`

This function takes the `history` object returned by the `model.fit` method and plots the training and validation accuracy and loss values over the epochs.

#### Plotting Accuracy

1. **Training Accuracy**: The accuracy of the model on the training data for each epoch.
2. **Validation Accuracy**: The accuracy of the model on the validation data for each epoch.
3. **Plot**: Both training and validation accuracy are plotted on the same graph. This helps visualize how the model's accuracy changes over time and how well it performs on training versus unseen validation data.

**Significance**:
- **Training Accuracy**: Indicates how well the model is learning from the training data.
- **Validation Accuracy**: Indicates how well the model generalizes to new, unseen data.
- **Comparison**: High training accuracy but low validation accuracy may indicate overfitting.

#### Plotting Loss

1. **Training Loss**: The loss (error) of the model on the training data for each epoch.
2. **Validation Loss**: The loss of the model on the validation data for each epoch.
3. **Plot**: Both training and validation loss are plotted on the same graph. This helps visualize how the model’s loss changes over time and how well it minimizes error on training versus validation data.

**Significance**:
- **Training Loss**: Indicates how well the model is fitting the training data.
- **Validation Loss**: Indicates how well the model is fitting the validation data.
- **Comparison**: Lower training loss compared to validation loss may indicate overfitting, while high values for both may indicate underfitting.

#### Usage

To use this function, call it after training your model by passing the `history` object. This will generate and display the plots for accuracy and loss, helping diagnose issues like overfitting or underfitting. The plots are also saved as an image file named `model_metrics.png` for further reference.

## Show Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Predict the classes for the validation data
validation_generator.reset()  # Ensure the generator is at the start
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(12, 12))

# Create color maps: green for correct and red for incorrect predictions
cmap_correct = plt.cm.Greens
cmap_incorrect = plt.cm.Reds

# Normalize the confusion matrix for color scaling
norm_conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Plot each cell
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        if i == j:
            color = cmap_correct(norm_conf_matrix[i, j])
        else:
            color = cmap_incorrect(norm_conf_matrix[i, j])
        ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=True, color=color, alpha=0.6))
        ax.text(j, i, str(conf_matrix[i, j]), va='center', ha='center', color='black', fontsize=12)

# Plot grid and labels
ax.matshow(np.zeros_like(conf_matrix), cmap=plt.cm.Blues, alpha=0)  # Invisible grid overlay for structure
ax.set_xticks(np.arange(len(class_labels)))
ax.set_yticks(np.arange(len(class_labels)))
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.set_yticklabels(class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Calculate scores for each label
scores = [(class_labels[i], conf_matrix[i, i], conf_matrix[i].sum() - conf_matrix[i, i]) for i in range(len(class_labels))]

# Sort scores by the number of correct predictions in descending order
scores.sort(key=lambda x: x[1], reverse=True)

# Create a legend with the sorted scores
legend_elements = [
    plt.Line2D([0], [0], color=cmap_correct(0.6), lw=4, label=f'{label} Correct: {correct}, Incorrect: {incorrect}')
    for label, correct, incorrect in scores
]

# Add the legend outside the plot
ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1), title="Scores")

# Save the figure
plt.savefig('tf_confusion_matrix.png', bbox_inches='tight')

# Show the plot
plt.show()


### Understanding the Confusion Matrix

A confusion matrix is a table used to evaluate the performance of a classification model. It provides a comprehensive summary of prediction results by comparing the actual and predicted classes. Here's a breakdown of what each element in the confusion matrix represents:

#### Structure of the Confusion Matrix

|                | Predicted Class 1 | Predicted Class 2 | ... | Predicted Class N |
|----------------|-------------------|-------------------|-----|-------------------|
| **Actual Class 1** | True Positive (TP)   | False Negative (FN) | ... | False Negative (FN) |
| **Actual Class 2** | False Positive (FP)  | True Positive (TP)   | ... | False Negative (FN) |
| ...            | ...               | ...               | ... | ...               |
| **Actual Class N** | False Positive (FP)  | False Positive (FP)  | ... | True Positive (TP)   |

#### Key Terms

1. **True Positive (TP)**: The number of instances correctly predicted as the positive class.
2. **True Negative (TN)**: The number of instances correctly predicted as the negative class.
3. **False Positive (FP)**: The number of instances incorrectly predicted as the positive class (Type I error).
4. **False Negative (FN)**: The number of instances incorrectly predicted as the negative class (Type II error).

#### Metrics Derived from the Confusion Matrix

- **Accuracy**: The overall correctness of the model, calculated as \((TP + TN) / (TP + TN + FP + FN)\).
- **Precision**: The accuracy of positive predictions, calculated as \(TP / (TP + FP)\).
- **Recall (Sensitivity)**: The ability of the model to find all the relevant cases (actual positives), calculated as \(TP / (TP + FN)\).
- **F1 Score**: The harmonic mean of precision and recall, providing a single metric that balances both, calculated as \(2 \times (Precision \times Recall) / (Precision + Recall)\).

#### Interpreting the Confusion Matrix

- **Diagonal Elements (TP and TN)**: Represent the correct predictions. High values indicate good model performance.
- **Off-Diagonal Elements (FP and FN)**: Represent the incorrect predictions. Low values are desirable.

By analyzing the confusion matrix, you can identify specific areas where your model performs well and areas where it needs improvement. For example, a high number of false negatives might indicate that the model is missing instances of a particular class, while a high number of false positives might suggest that the model is incorrectly predicting instances as that class.

Using the confusion matrix, you can fine-tune your model to improve its accuracy and reliability, especially for imbalanced datasets where certain classes may be underrepresented.

## Classify Test Images

In [ ]:
def classify_image(img_path, model_name,lbl_classes,target_size=(pic_size, pic_size)):
    """
    Classify a single image using the trained model.

    Parameters:
    - img_path: Path to the image to be classified
    - target_size: Size to which the image should be resized (default is (48, 48))

    Returns:
    - Predicted emotion label
    """
    import cv2
    from PIL import Image
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing import image
    # Load the trained model
    model = load_model(model_name)


    # Load and preprocess the image
    img = image.load_img(img_path, target_size=target_size, color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Scale pixel values to [0, 1]

    # Make a prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Get the emotion label
    emotion = lbl_classes[predicted_class]

    # Load the image using OpenCV for displaying
    img_cv = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Display the image and emotion prediction
    plt.imshow(img_cv, cmap='gray')
    plt.title(f'Prediction: {emotion}')
    plt.axis('off')
    plt.show()

    return emotion


# Loop through random images for predictions

In [ ]:
import os
import random 

def rand_images(test_path, model_name,lbl_classes):
    """
    Lists all .jpg images in the specified directory and applies the classify_image function to each.
    
    Args:
    directory (str): The directory to search for .jpg images.
    model_name: The model to use for image classification.
    """


    subdirs = [os.path.join(test_path, d) for d in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, d))]
    all_images = []

    for subdir in subdirs:
        images = [os.path.join(subdir, f) for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f)) and f.lower().endswith(('.jpg'))]
        all_images.extend([(subdir, img) for img in images])

    img = str(random.choice(all_images)[1])

    classify_image(img, model_name,lbl_classes)



In [ ]:
rand_images(validation_data_dir, model_name,lbl_classes)

In [ ]:
rand_images(validation_data_dir, model_name,lbl_classes)

In [ ]:
rand_images(validation_data_dir, model_name,lbl_classes)